ollama run llama3.2:1b

In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dspy

lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
dspy.configure(lm=lm)

In [5]:
from typing import Literal

class Categorize(dspy.Signature):
    """Classify historic events."""

    event: str = dspy.InputField()
    category: Literal[
        "Wars and Conflicts",
        "Politics and Governance",
        "Science and Innovation",
        "Cultural and Artistic Movements",
        "Exploration and Discovery",
        "Economic Events",
        "Social Movements",
        "Man-Made Disasters and Accidents",
        "Natural Disasters and Climate",
        "Sports and Entertainment",
        "Famous Personalities and Achievements"
    ] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Categorize)

# Here is how we call this module
classification = classify(event="World War II[b] or the Second World War (1 September 1939 – 2 September 1945) was a global conflict between two coalitions: the Allies and the Axis powers. Nearly all of the world's countries participated, with many nations mobilising all resources in pursuit of total war. Tanks and aircraft played major roles, enabling the strategic bombing of cities and delivery of the first and only nuclear weapons ever used in war. World War II was the deadliest conflict in history, resulting in 70 to 85 million deaths, more than half of which were civilians. Millions died in genocides, including the Holocaust, and by massacres, starvation, and disease. After the Allied victory, Germany, Austria, Japan, and Korea were occupied, and German and Japanese leaders were tried for war crimes.")
classification

Prediction(
    category='Wars and Conflicts',
    confidence=0.95
)

Trace(request_id=20b8973ecfc34a75aa181fafbd25859a)

In [6]:
classify(event="Second Boer War: In the Battle of Magersfontein the Boers commanded by general Piet Cronjé inflict a defeat on the forces of the British Empire commanded by Lord Methuen trying to relieve the Siege of Kimberley.")

Prediction(
    category='Wars and Conflicts',
    confidence=1.0
)

Trace(request_id=b829509e8a7a4fc491d51125f6329376)

In [18]:
import pandas as pd

with open('events.txt', 'r') as f:
    events = f.readlines()

def classify_event(event_text):
    result = classify(event=event_text)
    return result.category, result.confidence

events = pd.DataFrame(events, columns=['event_string'])

with dspy.context(lm=dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')):

    classifications = events['event_string'].apply(classify_event)

    events['category_32_1b'] = [result[0] for result in classifications]
    events['confidence_32_1b'] = [result[1] for result in classifications]

events

,event_string,category_32_1b,confidence_32_1b
0,627 – Battle of Nineveh: A Byzantine army unde...,Wars and Conflicts,2.00
1,1388 – Maria of Enghien sells the lordship of ...,Wars and Conflicts,1.00
2,1787 – Pennsylvania becomes the second state t...,Politics and Governance,0.95
3,1862 – American Civil War: USS Cairo sinks on ...,Wars and Conflicts,0.50
4,1866 – Oaks explosion: The worst mining disast...,Wars and Conflicts,0.55
5,1870 – Joseph H. Rainey of South Carolina beco...,Politics and Governance,0.50
6,1901 – Guglielmo Marconi receives the first tr...,Science and Innovation,0.95
7,1915 – Yuan Shikai declares the establishment ...,Wars and Conflicts,2.00
8,1917 – Father Edward J. Flanagan founds Boys T...,Wars and Conflicts,2.50
9,"1935 – The Lebensborn Project, a Nazi reproduc...",Wars and Conflicts,0.80


[Trace(request_id=7b7b978572a64a8988351de227994efe), Trace(request_id=a093817ba0c8433ebbc377de1090daf0), Trace(request_id=91155258ceaa47379e39b4e2efbac74f), Trace(request_id=699d3c0b405048b7839b4cbb32409ad1), Trace(request_id=4cbe7a42da0241388e38246825fcb621), Trace(request_id=58b54d03309442719347bbd265000e50), Trace(request_id=43555b02b0f943f384955e4760c8f081), Trace(request_id=c2e3dcccbac94e7698ab0243b522ead5), Trace(request_id=34536ef1d5da4286952de3fea78cfe5e), Trace(request_id=df85c0d8c0e045b6aef5c3275948b175)]

In [19]:
with dspy.context(lm=dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p3-70b-instruct')):

    classifications = events['event_string'].apply(classify_event)

    events['category_33_70b'] = [result[0] for result in classifications]
    events['confidence_33_70b'] = [result[1] for result in classifications]

events

,event_string,category_32_1b,confidence_32_1b,category_33_70b,confidence_33_70b
0,627 – Battle of Nineveh: A Byzantine army unde...,Wars and Conflicts,2.00,Wars and Conflicts,0.95
1,1388 – Maria of Enghien sells the lordship of ...,Wars and Conflicts,1.00,Politics and Governance,0.90
2,1787 – Pennsylvania becomes the second state t...,Politics and Governance,0.95,Politics and Governance,0.95
3,1862 – American Civil War: USS Cairo sinks on ...,Wars and Conflicts,0.50,Wars and Conflicts,0.95
4,1866 – Oaks explosion: The worst mining disast...,Wars and Conflicts,0.55,Man-Made Disasters and Accidents,1.00
5,1870 – Joseph H. Rainey of South Carolina beco...,Politics and Governance,0.50,Politics and Governance,0.95
6,1901 – Guglielmo Marconi receives the first tr...,Science and Innovation,0.95,Science and Innovation,0.95
7,1915 – Yuan Shikai declares the establishment ...,Wars and Conflicts,2.00,Politics and Governance,0.95
8,1917 – Father Edward J. Flanagan founds Boys T...,Wars and Conflicts,2.50,Social Movements,0.80
9,"1935 – The Lebensborn Project, a Nazi reproduc...",Wars and Conflicts,0.80,Politics and Governance,0.95


[Trace(request_id=ee151673e1194de1b51108bb21a159b3), Trace(request_id=d4933e369e794b6ca20f47075ca96d8b), Trace(request_id=04c98fdff7ed40a882cdf5714c99495e), Trace(request_id=5cc48772449c4c5694e7a2f67fd64bcc), Trace(request_id=a76a1746296c44a8a8c5f7fc41bbc80c), Trace(request_id=09402910ee1e4f8290a14260774e80d7), Trace(request_id=375802751a3049028944c818b5cec333), Trace(request_id=829fe61fea734318a3acedaa0626e58f), Trace(request_id=657858855d784aa4b7021cb36f8fecac), Trace(request_id=b8b1e50cf87346f39bb77f35b98a2a15)]